In [6]:
!apt-get install -y -qq glpk-utils
!pip install pyomo
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~MODEL~~~~~~~~~~~~~~~~~~~~~~~~~~~
# GLPK solver for non-linear programming with binary and integer variable
#    min    Sum_{k in K} c_k
#    s.t.   Sum_{k in K} p_k=3000
#           c1=2 * p[1]
#           c2= a * (2* p[2] +1000)
#           c3=7* p[3]
#           c4= 2*p[4]
#           0 <= p_k <= 1000
#           set k= {1,2,3,4}
#           p as integer
#           a as binary
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
import pyomo.environ as pyo
from pyomo.opt import SolverFactory
from pyomo.environ import *

#model
model = pyo.ConcreteModel()

#variable
model.c=pyo.Var(range(1,5))
model.p= pyo.Var(range(1,5) , within= Integers , bounds= (0,1000))
model.a= pyo.Var(within= Binary)
c= model.c
p= model.p
a=model.a
M= 100000

#objectivefunction
model.obj= pyo.Objective(expr= pyo.summation(c))

#constraints
model.total_p= pyo.Constraint(expr= pyo.summation(p) == 3000)
model.c1 = pyo.Constraint(expr=c[1] == 2 * p[1])

#model.c2 = pyo.Constraint(expr=c[2] == a *(2*p[2]+ 1000))
model.c2a= pyo.Constraint(expr= -a * M <= c[2] )
model.c2b= pyo.Constraint(expr= c[2] <= a* M )
model.c2c= pyo.Constraint(expr= -(1-a) * M <= c[2] - (2 * p[2]+1000) )
model.c2d= pyo.Constraint(expr= c[2] - (2* p[2]+1000) <= (1-a)* M)

model.c2p= pyo.Constraint(expr= p[2] <= a* 1000 )
model.c3= pyo.Constraint(expr= c[3] == 7* p[3])
model.c= pyo.Constraint(expr= c[4]== 2*p[4])

#solver
solver = SolverFactory('glpk', executable='/usr/bin/glpsol')  # مسیر اجرایی GLPK
solver.solve(model)
pyo.display(model)
#________________________________results________________________________
print('p1=', pyo.value(p[1]))
print('p2=', pyo.value(p[2]))
print('p3=', pyo.value(p[3]))
print('p4=', pyo.value(p[4]))
print('ptotal=', pyo.value(pyo.summation(p)))
print('\n------------------------------------------------------------')
print('c1=', pyo.value(c[1]))
print('c2=', pyo.value(c[2]))
print('c3=', pyo.value(c[3]))
print('c4=', pyo.value(c[4]))
print('ctotal=', pyo.value(pyo.summation(c)))

This is usually indicative of a modelling error.
To avoid this warning, use block.del_component() and block.add_component().


Model unknown

  Variables:
    p : Size=4, Index=p_index
        Key : Lower : Value  : Upper : Fixed : Stale : Domain
          1 :     0 : 1000.0 :  1000 : False : False : Integers
          2 :     0 : 1000.0 :  1000 : False : False : Integers
          3 :     0 :    0.0 :  1000 : False : False : Integers
          4 :     0 : 1000.0 :  1000 : False : False : Integers
    a : Size=1, Index=None
        Key  : Lower : Value : Upper : Fixed : Stale : Domain
        None :     0 :   1.0 :     1 : False : False : Binary

  Objectives:
    obj : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 6999.99999999999

  Constraints:
    total_p : Size=1
        Key  : Lower  : Body   : Upper
        None : 3000.0 : 3000.0 : 3000.0
    c1 : Size=1
        Key  : Lower : Body : Upper
        None :   0.0 :  0.0 :   0.0
    c2a : Size=1
        Key  : Lower : Body                : Upper
        None :  None : -102999.99999999999 :   0.0
    c2b : Size=1
     